##### IMPORTAR AS PLANILHAS

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

##### IMPORTANDO PLANILHA

In [ ]:
path_iw = "../../../data/util/os/"

df_iw49 = pd.read_csv(path_iw + "IW47_Executadas_processado.csv")

In [ ]:
df_iw49["Data fim"] = pd.to_datetime(df_iw49["Data fim"])
df_iw49["Data inicio"] = pd.to_datetime(df_iw49["Data inicio"])
df_iw49["Local de instalação"] = df_iw49["Local de instalação"].astype(str)

##### CRIANDO DATAFRAME SEM INCLUIR DATAS FINAIS ANTERIORES A DATAS INICIAIS

In [ ]:
# Removendo linhas onde a data final é superior a data inicial
df_sem_linhas_irregulares = df_iw49.loc[df_iw49["Data fim"]
                                        >= df_iw49["Data inicio"]]

num_linhas_removidas = df_iw49.shape[0] - df_sem_linhas_irregulares.shape[0]
print("Linhas removidas:", num_linhas_removidas)

##### FUNÇÃO QUE ADD COLUNA DE HH POR MES E QUANTIDADE DE MESES PASSADOS DURANTE A OS

In [ ]:
# Soma dos meses, em funcao da quantidade de anos que se passaram e meses, entre o inicio e fim da OS
df = df_sem_linhas_irregulares

df["Meses"] = (
    (df["Data fim"].dt.year - df["Data inicio"].dt.year) * 12
    + (df["Data fim"].dt.month - df["Data inicio"].dt.month)
) + 1
df["HH por mes"] = df["HH final"] / df["Meses"]

##### FUNCAO QUE AGRUPA OS POR ANO, MES, LOCAL

In [ ]:
df["Data inicio"] = df["Data inicio"].apply(lambda x: x.replace(day=1))
df["Data fim"] = df["Data fim"].apply(lambda x: x.replace(day=1))

In [ ]:
# criar subselecoes do df escolhendo todas as OP ativas em cada mes em cada LI
df_agrupado = pd.DataFrame(
    columns=["Ano Mes", "Local de instalação", "HH"])

for local_instalacao in df["Local de instalação"].unique():
    no_local = df[df["Local de instalação"] == local_instalacao]

    data_inicio_min = no_local["Data inicio"].min()
    data_fim_max = no_local["Data fim"].max()
    espaco_de_tempo = pd.date_range(
        start=data_inicio_min, end=data_fim_max, freq="MS")

    for ano_mes in espaco_de_tempo:
        ativas = no_local[(no_local["Data inicio"] <= ano_mes)
                          & (no_local["Data fim"] >= ano_mes)]
        if ativas.shape[0] == 0:
            continue
        hh_por_mes_total = ativas["HH por mes"].sum()
        df_agrupado.loc[len(df_agrupado)] = [
            ano_mes, local_instalacao, hh_por_mes_total]

##### TRATAMENTO FINAL PARA PODER EXPORTAR A TABELA

In [ ]:
df_agrupado = df_agrupado.sort_values(
    by=["Ano Mes", "Local de instalação"]).reset_index(drop=True)
df_agrupado['Ano Mes'] = df_agrupado['Ano Mes'].astype(str).str[:7]
df_agrupado['Ano Mes'].unique()

##### EXPORTANDO PLANILHA

In [ ]:
df_agrupado.to_csv(path_iw + "os_entrada.csv", index=False)